# Instrucciones

Conectar el mando al Jetracer, configurar constantes y ejecutar el cuaderno.

La pantalla del Jetracer muestra los ajustes de velocidad máxima / desplazamiento de la dirección.

## Controles

* **`Joystick izquierdo`**: velocidad
* **`Joystick derecho`**: dirección
* **`Y`/`A`**: velocidad máxima
* **`B`/`X`**: corrección desplazamiento dirección
* **`LT`**: graba vídeo / detiene grabación
* **`SELECT`**: modo conducción manual
* **`START`**: modo conducción IA
* **`LT2 + RT2`**: sale del bucle de control (por si queremos hacer cambios en el cuaderno y reejecutar)

# TODO
- Evento "DESCONEXIÓN" del mando?
- Poder hacer cambios en el cuaderno sin tener que reiniciar kernel


In [1]:
from datetime import datetime
import time
import threading

import cv2
import ipywidgets
import numpy as np
import requests
import torch
import torchvision
from inputs import get_gamepad
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
from jetracer.nvidia_racecar import NvidiaRacecar
from matplotlib import pyplot as plt
from torch2trt import TRTModule

from pathlib import Path

# Constants

* **`STEERING_OFFSET`**: usar el modo de conducción manual y los botones `X`/`B` para corregir el error en la dirección según el valor mostrado en la pantalla del Jetracer.
* **`FPS`**: recomendado 15 para conducción IA y grabación de vídeo simultánea
* **`THROTTLE_GAIN`**: recomendado 0.15 para la pista oval de Waveshare
* **`PREVIEW`**: muestra predicciones en tiempo real en Jupyer Lab
* **`MODEL`**: nombre del modelo. Debe estar en el directorio `models`. Sin sufijo `_trt.pth`

In [2]:
CAP_WIDTH = 640 #960
CAP_HEIGHT = 480 #540
SZ = 224
FPS = 1
THROTTLE_GAIN = 0.15
STEERING_OFFSET = +0.15
STEERING_GAIN = -0.55
PREVIEW = False
MODEL = 'cida_javi_cristal_ep15_lr3e-4'
OUT_P = Path('train_dataset') # directorio de destino de las fotos (se creará)

# Read pretrained model

# Create RaceCar object

In [3]:
car = NvidiaRacecar()

# Create camera object

Si sale el siguiente error:

```
RuntimeError: Could not read image from camera.
```

Abrir un terminal desde File -> New -> Terminal, ejecutar:

```
sudo systemctl restart nvargus-daemon
```

Después, Kernel -> Restart kernel

In [4]:
#camera = CSICamera(width=SZ, height=SZ, capture_fps=30) # fps=65
camera = CSICamera(
    capture_width=CAP_WIDTH, 
    capture_height=CAP_HEIGHT,
    width=CAP_WIDTH, 
    height=CAP_HEIGHT, 
    capture_fps=FPS) # fps=65

In [5]:
#image = camera.read()

In [6]:
#plt.imshow(image)

In [7]:
#plt.imshow(image)

# Display functions

In [5]:
def update_display():
    global manual_drive, video_out
    info = '\n'.join([
        f"th_gain: {car.throttle_gain:.2f}",
        f"st_offs: {car.steering_offset:.2f}",
        f"{'RECORDING VIDEO' if video_out is not None else ''}",
        f"{'MANUAL' if manual_drive else 'AI'} drive",
    ])
    requests.get("http://localhost:8000/text/" + info)

def reset_display():
    requests.get("http://localhost:8000/stats/on")

# Video recording functions

In [6]:
def open_mov():
    global video_out
    fn = datetime.utcnow().strftime('%Y%m%d_%H%M%S_%f') + '.mov'
    gst_out = f"appsrc ! video/x-raw, format=BGR ! queue ! videoconvert ! video/x-raw,format=RGBA ! nvvidconv ! nvv4l2h264enc ! h264parse ! qtmux ! filesink location={fn} " # trailing space is REQUIRED
    video_out = cv2.VideoWriter(gst_out, cv2.CAP_GSTREAMER, 0, float(FPS), (CAP_WIDTH, CAP_HEIGHT))
    if not video_out.isOpened():
        print("Failed to open output")
    #print("open_mov")

def close_mov():
    global video_out
    video_out.release()
    video_out = None
    #print("close_mov")

# Driving (AI / manual) thread

In [10]:
preview_widget = ipywidgets.Image(width=CAP_WIDTH, height=CAP_HEIGHT)

if PREVIEW:
    display(preview_widget)

In [11]:
def get_road_center(image):
    h, w = image.shape[:2]
    image = cv2.resize(image, (SZ, SZ))
    
    # BGR -> RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert uint8 array to float tensor, div by 255, permute dimensions HWC to CHW
    t = torchvision.transforms.functional.to_tensor(image) # uint8 -> float, [0,255] -> [0,1], 3, SZ, SZ
    
    # Move to GPU
    t = t.cuda()
    
    # Normalize with imagenet stats since we're using an imagenet pretrained model
    t = (t - mean[:,None,None]) / std[:,None,None]
    
    # Add batch dimension
    t = t[None, ...] # 1, 3, SZ, SZ
    
    # float -> float16 for faster inference
    t = t.half()

    # Get center of road
    o = model_trt(t)

    # Clamp to [-1, 1]
    o = torch.clamp(o, min=-1, max=1)

    # Remove batch dimension
    o = o.flatten()
    
    # Move to CPU
    o = o.cpu()
    
    # Get x, y values between -1, 1
    nx, ny = o.tolist()

    # Map to image x and y
    ix = int((nx + 1) / 2 * (w - 1))
    iy = int((ny + 1) / 2 * (h - 1))

    return nx, ny, ix, iy

In [12]:
#%%timeit
if False:
    image = camera.read()

    # Get center of road
    x, y, ix, iy = get_road_center(image)

    # Draw circle and preview
    cv2.circle(image, (ix, iy), 8, (0, 255, 0), 2)

    jpg = bgr8_to_jpeg(image)
    preview_widget.value = jpg

In [9]:
def update_image(change):
    # New image is a numpy array with shape (SZ, SZ, 3), dtype uint8, BGR channels
    image = change['new']

    # Get center of road
    x, y, ix, iy = get_road_center(image)

    # AI drive
    if not manual_drive:
        car.steering = x
    
    # Draw circle
    cv2.circle(image, (ix, iy), 8, (0, 255, 0), 2)
    
    if PREVIEW:
        jpg = bgr8_to_jpeg(image)
        preview_widget.value = jpg
    
    if video_out is not None:
        #print("Write frame")
        video_out.write(image)
        

# Driving loop

In [10]:
right_trigger = left_trigger = 0
car.throttle = 0.0
car.throttle_gain = THROTTLE_GAIN
car.steering_offset = STEERING_OFFSET
car.steering_gain = STEERING_GAIN
manual_drive = True
video_out = None

In [11]:
camera.observe(update_image, names='value')

In [13]:
camera.running = True

In [14]:
def get_timestamp():
    return datetime.utcnow().strftime('%Y%m%d_%H%M%S_%f')

In [15]:
right_trigger = left_trigger = 0
while not (right_trigger and left_trigger):
    events = get_gamepad()
    for event in events:
        print(event.ev_type, event.code, event.state)
        if event.ev_type == 'Absolute' and manual_drive:
            if event.code == 'ABS_X':
                #car.throttle = -(event.state - 127.5) / 127.5
                print(f'X={event.state}')
            if event.code == 'ABS_Y':
                #car.throttle = -(event.state - 127.5) / 127.5
                print(f'Y={event.state}')
            if event.code == 'ABS_Z':
                #car.steering = (event.state - 127.5) / 127.5
                print(f'Z={event.state}')
        elif event.ev_type == 'Key' and event.code == 'BTN_TR2':
            right_trigger = event.state
        elif event.ev_type == 'Key' and event.code == 'BTN_TL2':
            left_trigger = event.state
        elif event.ev_type == 'Key' and event.state == 1:
            if event.code == 'BTN_WEST': # Y / UP
                #car.throttle_gain = min(1.0, car.throttle_gain + 0.05)
                #update_display()
                print('BTN_WEST')
            if event.code == 'BTN_SOUTH': # A / DOWN
                #car.throttle_gain = max(0.0, car.throttle_gain - 0.05)
                #update_display()
                print('BTN_SOUTH')
            if event.code == 'BTN_EAST': # B / RIGHT
                #car.steering_offset = max(-0.3, car.steering_offset - 0.05)
                #car.steering = 0.1
                #car.steering = 0
                #update_display()
                print('BTN_EAST')
            if event.code == 'BTN_NORTH': # X / LEFT
                print('FOTO')
                image = change['new']
                jpg = bgr8_to_jpeg(image)
                image_widget.value = jpg
                jpg_p = OUT_P / (get_timestamp() + ".jpg")
                with jpg_p.open('wb') as f:
                    f.write(jpg)
            if event.code == 'BTN_TL':
                print('BTN_TL')


        #else:
            #print(event.ev_type, event.code, event.state)
   
            #print(f"throttle_gain={car.throttle_gain:.1f}")
        #print(event.ev_type, event.code, event.state)
        
#manual_drive = True
#camera.running = False
#camera.unobserve_all()
#car.throttle = 0

#reset_display()

Absolute ABS_X 128
X=128
Absolute ABS_Y 127
Y=127
Absolute ABS_Z 128
Z=128
Absolute ABS_RZ 127
Sync SYN_REPORT 0
Absolute ABS_X 127
X=127
Absolute ABS_Y 128
Y=128
Absolute ABS_Z 127
Z=127
Absolute ABS_RZ 128
Sync SYN_REPORT 0
Misc MSC_SCAN 589828
Key BTN_NORTH 1
FOTO


NameError: name 'change' is not defined

In [ ]:
car.throttle=0

In [ ]:
car.steering=0